In [167]:
import os
import uuid
from typing import TypedDict, Sequence, Optional, Type, Annotated

from dotenv import load_dotenv
from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph, add_messages
from pydantic import Field, BaseModel
from pymongo import MongoClient
from langchain_core.tools import BaseTool

load_dotenv()

True

In [57]:
model = ChatOpenAI(model="gpt-4o")

In [58]:
client = MongoClient(os.environ["DEV_MONGO_CONNECTION_STRING"])

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)

vector_store = MongoDBAtlasVectorSearch(
    collection=client["lab_dev"]["langchain_embedding"],
    index_name="test_vector_store_index",
    relevance_score_fn="cosine",
    embedding=embeddings,
)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20},
)

In [59]:
class ContentSearchInput(BaseModel):
    query: str = Field(..., description="query string")

class ContentSearchTool(BaseTool):
    name: str = "ContentSearch"
    description: str = "search for places given a detailed query"
    args_schema: Type[BaseModel] = ContentSearchInput
    return_direct: bool = True

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> list[Document]:
        return retriever.invoke(query)
    
content_search_tool = ContentSearchTool()

In [60]:
model_with_tools = model.bind_tools([content_search_tool])

In [61]:
system_prompt = SystemMessagePromptTemplate.from_template("""
You are an AI concierge designed to assist users in planning their activities.
When a user provides a request, engage them in a conversation to clarify the details. Use the following steps:
	-	Greet the user warmly and acknowledge their request.
	-	You need to first clarify the following. Ask questions one at a time in a conversational manner (without listing or numbering) if needed.
	    -	Who is involved? (e.g., Are they alone, with a partner, friends, or family?)
        -	Where do they want to do it? (e.g., Region boundary?)
        -	What do they want to do? (e.g., Do they want to eat, walk or drink?)
        -	When do they want to do it? (e.g., Specific date, time, or any flexible preferences?)
    -   Once enough information is gathered, use "Content Search" tool to search for places given a detailed query.
    -   After search, answer using the provided context only.
    -   Suggest utmost 3 places in format of "name: reason".
    -   If you cannot find reasonable places, suggest spaces nearby and recommend uploading posts himself/herself.
	-	Use their answers to suggest tailored options, ensuring your responses are polite, empathetic, and helpful.
	-   You must answer in {language}
""")

prompt_template = ChatPromptTemplate.from_messages([
    system_prompt,
    MessagesPlaceholder(variable_name="messages"),
])

In [156]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
    
workflow = StateGraph(state_schema=State)

def call_model(state: State):
    input_messages = prompt_template.invoke(state)
    output_messages = []
    
    ai_message = model_with_tools.invoke(input_messages)
    output_messages = add_messages(output_messages, ai_message)
    
    if len(ai_message.tool_calls) > 0:
        tool_call = ai_message.tool_calls[0]
        tool_message = content_search_tool.invoke(tool_call)
        output_messages = add_messages(output_messages, tool_message)

        ai_message = model_with_tools.invoke(add_messages(input_messages.to_messages(), output_messages))
        output_messages = add_messages(output_messages, ai_message)
        
    return {"messages": output_messages}
        
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [157]:
def prompt(thread_id: str):
    message = input()
    print("You: " + message)
    print()
        
    streamer = app.stream(
        {"messages": [HumanMessage(message)], "language": "Korean"},
        {"configurable": {"thread_id": thread_id}},
        stream_mode="messages"
    )

    print("Bot: ")
    for chunk, metadata in streamer:
        if isinstance(chunk, AIMessage):
            print(chunk.content, end="")

In [158]:
thread_id = str(uuid.uuid4())

In [159]:
prompt(thread_id)

You: 오늘밤에 친구랑 단둘이 술먹기좋은 이자카야 추천해줘!

Bot: 
안녕하세요! 오늘밤에 친구분과 함께 가실 이자카야를 찾고 계시군요. 어디에서 즐기고 싶으신가요? 특정 지역이 있으신가요?

In [160]:
prompt(thread_id)

You: 서울 어디든 좋아~

Bot: 
좋습니다! 서울에서 친구분과 함께 할 수 있는 멋진 이자카야를 찾아드리겠습니다. 잠시만 기다려 주세요.서울에서 친구와 함께 술을 즐기기 좋은 이자카야를 추천해드릴게요.

1. 카사이: 서초구에 위치한 현대적인 이자카야로, 깔끔하고 세련된 인테리어가 매력적입니다. 다양한 일본식 요리를 맛볼 수 있으며, 특히 네기토로 핸드롤과 매콤한 해물 나베가 인기입니다.

2. 지하서점: 성북구에 위치한 독특한 바입니다. 아늑한 도서관 분위기와 활기찬 바의 조화가 매력적이며, 책과 함께 맥주를 즐기며 조용히 시간을 보내기에 좋습니다.

3. 오카츠: 성동구에 위치한 일본 전통 요리 전문점으로, 특히 돈카츠가 유명합니다. 깔끔한 인테리어와 맛있는 요리로 친구와 편안한 시간을 보내기에 적합합니다.

이 중에서 마음에 드는 곳이 있으신가요? 추가 도움이 필요하시면 말씀해 주세요!